In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [4]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

# Preprocessing

In [5]:
text=data["transcription_text"]

In [6]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [7]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [9]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:02<00:00,  9.36it/s]


In [10]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 3061.54it/s]


In [11]:
text[1]

'I E C Whaley solemnly swear I carefully read Amnesty Proclamation issued Andrew Johnson President of United State of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made except E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben Maj 47th PM United States of America E C Whaley of city county Charleston Sate of South Ca solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of SlavesSO HELP ME GOD E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben May 47 PM The fair complexion light hair gray eye 5 foot 10 inch high 39 year profession planter Copy Levi Steuben Maj 47th Pro Marshal City of Charleston'

In [12]:
text[10]

'AMNESTY OATH Mississippi City Miss Sept 29 1865 I JW Baxter solemnly swear presence of Almighty God I henceforth faithfully support protect defend Constitution of United States union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference emancipation of slave SO HELP ME GOD JW Baxter Sworn subscribed Mississippi City 29th day of September 1865 J H Mathews Capt 66th US Col Infty Provost Marshal Sub port Miss DESCRIPTION Age 24 year Height 5ft 8 inch Eyes Blue Color of Hair light Characteristics HEADQUARTERS Mississippi City Miss Sept 29th 1865 I hereby certify Amnesty Oath prescribed Proclamation of President of United States duly administered JW Baxter of Harrison Co order 29th day of Sept 29 1865 Frank E Miller Maj 66th USCI Commanding'

# Flair

In [13]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [14]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.22 MiB/s, done.


In [15]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-07 15:02:09,780 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-07 15:02:28,930 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [16]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-07 15:02:48,192 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-07 15:02:50,715 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [17]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-07 15:04:05,907 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-07 15:04:21,864 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [18]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(20):
        sentencess =Sentence(text[i])
        dflarge.loc[i,"text"]=text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [19]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa..."
3,No 2324 I solemnly swear affirm presence of Al...,"[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]"
5,I Wm Whaley solemnly swear I carefully read Am...,"[Wm Whaley, ANDREW JOHNSON, William Whaley, Le...","[United States of America, Charleston, SC, Cit...","[Amnesty Proclamation, Constitution]"
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...","[United States of America, Charleston Sc, Coun...","[Amnesty Proclamation, Constitution, Emancipat..."
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Joseph Edings ...","[United States of America, County of Orangebur...","[Amnesty Proclamation, NYVV, Constitution]"
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[],[],[Thomas Andrew Oath of Loyalty Recd Bureau]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, R A Bray, Alm...","[United States of America, UNITED STATES OF AM...","[Amnesty Proclamation, Walhalla, Constitution,..."


tagger1 ner english

In [20]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dfner=pd.DataFrame({"text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(text[i])
        dfner.loc[i,"text"]=text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dfner["PERSON-nerenglish"]=Per
dfner["LOC-nerenglish"]=LOC
dfner["ORG-nerenglish"]=ORG                        

In [21]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...


tagger 2 ontonotes

In [22]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

df=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(20):
        sentencess =Sentence(text[i])
        df.loc[i,"text"]=text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
df["Date-ontonotes"]=Date
df["PERSON-ontonotes"]=Per
df["GPE-ontonotes"]=GPE
df["ORG-ontonotes"]=ORG    

In [23]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865]


****

# Post processing

**For dates**

In [24]:
df[:2]

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ..."


In [25]:
date2=re.search(r'^[A-Z]\.*.*\d \.*.*\d\d$' ,"May 29th 1865")
date2

In [26]:

G=[]
for i in df["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
            
            

 

In [27]:
GF


[['May 29th 1865', '27th day of Novb 1865', '27th day of Novbr 1865'],
 ['May 29th 1865', '28 day of Sept 1865'],
 ['May 29th 1865', '31 day of Oct 1865'],
 ['31st day of August AD 1865'],
 ['August 24 1865'],
 ['May 29th 1865', '3 day of August 1865', '3 day of Augt 1865'],
 ['May 29th 1865', '16th day of Sept 1865'],
 ['May 29th 1865', '16 day of October 1865'],
 [],
 ['May 29th 1865', '6 day of Nov 1865'],
 ['Sept 29 1865',
  '29th day of September 1865',
  'Sept 29th 1865',
  '29th day of Sept 29 1865'],
 ['3d day of February 1868'],
 [],
 ['May 29th 1865', '7th day of September 186'],
 ['Aug 20 1865', '20th day of August 1865', 'Aug 20th 1865'],
 ['August 29th 1865', '22nd day of August 1865'],
 [],
 ['18th Sept 1865'],
 [],
 ['Aug 31 1865', '31st day of Aug 1865', '31 day of Aug 1865']]

In [28]:
df["postDate-ontonotes"]=GF

In [29]:
df.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[May 29th 1865, 27th day of Novb 1865, 27th da..."
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[May 29th 1865, 28 day of Sept 1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[May 29th 1865, 31 day of Oct 1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],[31st day of August AD 1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[August 24 1865]


In [30]:
G=[]
for i in df["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z]\.*.*)\s([0-9]{3,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{3,4})',r'\2-\1-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
    G.append(M)
    



In [31]:
df["postDate-ontonotes"]=G

In [32]:
df

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27-Novbr-1865]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],[31-AD-1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865]
5,I Wm Whaley solemnly swear I carefully read Am...,"[Whaley, ANDREW JOHNSON, William Whaley Sworn,...","[United States of America, Charleston SC, Char...",[],"[May 29th 1865, 3 day of August 1865, May 4, 3...","[29-May-1865, 3-August-1865, 3-Augt-1865]"
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...",[United States of America],[],"[May 29th 1865, 16th day of Sept 1865, 58 year]","[29-May-1865, 16-Sept-1865]"
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Frank Geise]","[United States of America, Orangeburg, South C...",[],"[May 29th 1865, 16 day of October 1865, twenty...","[29-May-1865, 16-October-1865]"
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[Vicksburg],[],[],[July 22],[]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, Jacob Schrode...","[United States of America, UNITED STATES OF]",[Walhalla],"[May 29th 1865, 6 day of Nov 1865, 58 year of ...","[29-May-1865, 6-Nov-1865]"


**great !**

# best result obtained 

***Person***

****

***Organization***

****

***GPE and location***

****

**Date**

****

In [33]:
df

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27-Novbr-1865]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]"
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]"
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],[31-AD-1865]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865]
5,I Wm Whaley solemnly swear I carefully read Am...,"[Whaley, ANDREW JOHNSON, William Whaley Sworn,...","[United States of America, Charleston SC, Char...",[],"[May 29th 1865, 3 day of August 1865, May 4, 3...","[29-May-1865, 3-August-1865, 3-Augt-1865]"
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...",[United States of America],[],"[May 29th 1865, 16th day of Sept 1865, 58 year]","[29-May-1865, 16-Sept-1865]"
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Frank Geise]","[United States of America, Orangeburg, South C...",[],"[May 29th 1865, 16 day of October 1865, twenty...","[29-May-1865, 16-October-1865]"
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[Vicksburg],[],[],[July 22],[]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, Jacob Schrode...","[United States of America, UNITED STATES OF]",[Walhalla],"[May 29th 1865, 6 day of Nov 1865, 58 year of ...","[29-May-1865, 6-Nov-1865]"


In [34]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]"
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat..."
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa..."
3,No 2324 I solemnly swear affirm presence of Al...,"[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]"
5,I Wm Whaley solemnly swear I carefully read Am...,"[Wm Whaley, ANDREW JOHNSON, William Whaley, Le...","[United States of America, Charleston, SC, Cit...","[Amnesty Proclamation, Constitution]"
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...","[United States of America, Charleston Sc, Coun...","[Amnesty Proclamation, Constitution, Emancipat..."
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Joseph Edings ...","[United States of America, County of Orangebur...","[Amnesty Proclamation, NYVV, Constitution]"
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[],[],[Thomas Andrew Oath of Loyalty Recd Bureau]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, R A Bray, Alm...","[United States of America, UNITED STATES OF AM...","[Amnesty Proclamation, Walhalla, Constitution,..."


In [35]:
dfner

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...
5,I Wm Whaley solemnly swear I carefully read Am...,[Wm Whaley],[So Carolina],[]
6,I J F Steinmeyer solemnly swear I carefully re...,[],[],[]
7,I Joseph Edings solemnly swear I carefully rea...,[Joseph Edings],[South Carolina],[]
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[],[],[]
9,455 I Jacob Schroder solemnly swear I carefull...,[Jacob Schroder],[AMERICA],[]


In [36]:
df["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
df["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
df["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]
df["Person-NerEng"]=dfner["PERSON-nerenglish"]
df["LOC-NerEng"]=dfner["LOC-nerenglish"]
df["ORG-NerEng"]=dfner["ORG-nerenglish"]

In [37]:
df

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,8 E encl I Anne H Elliott widow solemnly swear...,"[Anne H Elliott, Andrew Johnson, S Wilde Durson]",[United States of America],[Amnesty],"[May 29th 1865, 27th day of Novb 1865, 27th da...","[29-May-1865, 27-Novb-1865, 27-Novbr-1865]","[Anne H Elliott, Andrew Johnson, S Wilde Durso...","[United States of America, District of Beaufor...","[Amnesty, Constitution, Emancipation of Slaves]","[Anne H Elliott, Andrew Johnson, Anne H]",[South Carolina],[]
1,I E C Whaley solemnly swear I carefully read A...,"[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, Charleston SC, Unite...",[],"[May 29th 1865, 28 day of Sept 1865, Maj 47th ...","[29-May-1865, 28-Sept-1865]","[I E C Whaley, Andrew Johnson, E C Whaley, Lev...","[United State of America, United States of Ame...","[Amnesty Proclamation, Constitution, Emancipat...",[Andrew Johnson],[],[]
2,43M I Mrs E Montgomery solemnly Swear I carefu...,"[E, ANDREW JOHNSON, Levi Stuber Major]","[United States of America, Charleston SC, Unit...",[Union of States],"[May 29th 1865, 31 day of Oct 1865, 61 year]","[29-May-1865, 31-Oct-1865]","[Mrs E Montgomery, ANDREW JOHNSON, Sgd Montgom...","[United States of America, Charleston, SC, Cou...","[Amnesty Proclamation, 47th Pa, Vols, Emancipa...",[E Montgomery],[],[]
3,No 2324 I solemnly swear affirm presence of Al...,[Hu Kennedy],[],[],[31st day of August AD 1865],[31-AD-1865],"[God, GOD, EB Towne, Hu Kennedy]","[United States Union of States, City of New Or...",[Constitution],[],[],[]
4,August 24 1865 APPLICATION AND CERTIFICATE OF ...,[JB Hiserodt],[US],[],[August 24 1865],[24-August-1865],[JB Hiserodt],[Pass Christian Mississippi],"[WA Fleitas Amnesty Oath, US Cold Infty]",[],[US],[WA Fleitas Amnesty Oath Administered At Pass ...
5,I Wm Whaley solemnly swear I carefully read Am...,"[Whaley, ANDREW JOHNSON, William Whaley Sworn,...","[United States of America, Charleston SC, Char...",[],"[May 29th 1865, 3 day of August 1865, May 4, 3...","[29-May-1865, 3-August-1865, 3-Augt-1865]","[Wm Whaley, ANDREW JOHNSON, William Whaley, Le...","[United States of America, Charleston, SC, Cit...","[Amnesty Proclamation, Constitution]",[Wm Whaley],[So Carolina],[]
6,I J F Steinmeyer solemnly swear I carefully re...,"[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...",[United States of America],[],"[May 29th 1865, 16th day of Sept 1865, 58 year]","[29-May-1865, 16-Sept-1865]","[I J F Steinmeyer, ANDREW JOHNSON, JF Steinmey...","[United States of America, Charleston Sc, Coun...","[Amnesty Proclamation, Constitution, Emancipat...",[],[],[]
7,I Joseph Edings solemnly swear I carefully rea...,"[Joseph Edings, ANDREW JOHNSON, Frank Geise]","[United States of America, Orangeburg, South C...",[],"[May 29th 1865, 16 day of October 1865, twenty...","[29-May-1865, 16-October-1865]","[Joseph Edings, ANDREW JOHNSON, Joseph Edings ...","[United States of America, County of Orangebur...","[Amnesty Proclamation, NYVV, Constitution]",[Joseph Edings],[South Carolina],[]
8,Vicksburg Miss July 22 1867 Thomas Andrew Oath...,[Vicksburg],[],[],[July 22],[],[],[],[Thomas Andrew Oath of Loyalty Recd Bureau],[],[],[]
9,455 I Jacob Schroder solemnly swear I carefull...,"[Jacob Schroder, ANDREW JOHNSON, Jacob Schrode...","[United States of America, UNITED STATES OF]",[Walhalla],"[May 29th 1865, 6 day of Nov 1865, 58 year of ...","[29-May-1865, 6-Nov-1865]","[Jacob Schroder, ANDREW JOHNSON, R A Bray, Alm...","[United States of America, UNITED STATES OF AM...","[Amnesty Proclamation, Walhalla, Constitution,...",[Jacob Schroder],[AMERICA],[]


In [38]:
df.to_csv("result.csv", index=False)